In [12]:
print("hello, world!")

hello, world!


In [13]:
# system packages 
import os
import json

# internal packages 

# external packages
from openai import OpenAI
from dotenv import load_dotenv
from graphql import build_schema, parse
from jinja2 import Environment, FileSystemLoader

load_dotenv()
open_ai_api_key = os.getenv("OPENAI_API_KEY")

## GraphQL Schema Parsing

In [14]:

# with open('../assets/schemas/opensea/opensea_original_schema.graphql', 'r') as schema_file:
#     schema_str = schema_file.read()

# # TEMP: Try to parse the schema given the errors we are seeing
# custom_definitions = '''
# scalar BigDecimal
# scalar BigInt

# directive @entity on OBJECT
# directive @dailySnapshot on OBJECT
# directive @regularPolling on OBJECT
# directive @derivedFrom on OBJECT
# directive @transaction on OBJECT
# '''

# full_schema_str = custom_definitions + schema_str
# schema = build_schema(full_schema_str)

# for type_name, graphql_type in schema.type_map.items():
#     if type_name.startswith('__'):
#         continue  # Skip introspection types
#     print(f"Type: {type_name}")
#     for field_name, field in graphql_type.fields.items():
#         print(f"  Field: {field_name} (type: {field.type})")

## LangGraph

In [15]:
# import getpass
# import os

# if "OPENAI_API_KEY" not in os.environ:
#     os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
# open_ai_api_key = os.environ["OPENAI_API_KEY"]

In [16]:
# from typing import Annotated

# from langchain_openai import OpenAI
# from typing_extensions import TypedDict

# from langgraph.graph import StateGraph
# from langgraph.graph.message import add_messages


# class State(TypedDict):
#     messages: Annotated[list, add_messages]


# graph_builder = StateGraph(State)


# llm = OpenAI(model="gpt-4")


# def chatbot(state: State):
#     return {"messages": [llm.invoke(state["messages"])]}

# graph_builder.add_node("chatbot", chatbot)
# graph_builder.set_entry_point("chatbot")
# graph_builder.set_finish_point("chatbot")
# graph = graph_builder.compile()

In [17]:
# from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph().draw_mermaid_png()))
# except Exception:
#     pass

## OpenAI Directly

In [18]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)

def gpt_chat_completion(client, message, model="gpt-4o"):
    return client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": message,
        }
    ],
    model=model,
)

print(gpt_chat_completion(client, "Hello! How are you?").choices[0].message.content)

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


## Initial Test Set 

### Prompts 

- [ ] One entity quality vs. another entity quality 
- [ ] One column quality vs. another column quality 
- [ ] One schema quality vs. another schema quality 

### Demonstrations 

- [ ] A high quality entity (description)
- [ ] A high quality column (description)
- [ ] A high quality schema (description)

### Quality Categories

- [ ] 

In [19]:
with open("tests/assets/entity_comparison_assets.json", "r") as f:
    entity_comparison_assets = json.load(f)

gold_entity_comparison = "".join(entity_comparison_assets["gold"]["prompt"])
four_entity_comparison = "".join(entity_comparison_assets["four"]["prompt"])
three_entity_comparison = "".join(entity_comparison_assets["three"]["prompt"])
two_entity_comparison = "".join(entity_comparison_assets["two"]["prompt"])
one_entity_comparison = "".join(entity_comparison_assets["one"]["prompt"])

In [22]:
# Function to reload the template
def reload_template(template_path):
    env = Environment(loader=FileSystemLoader(template_path))
    env.cache.clear()
    entity_comparison_prompt_template = env.get_template("entity_comparison_prompt.txt")
    return entity_comparison_prompt_template

template_path = "../assets/prompts"
entity_comparison_prompt_template = reload_template(template_path)

# env = Environment(loader=FileSystemLoader("../assets/prompts"))
# entity_comparison_prompt_template = env.get_template("entity_comparison_prompt.txt")
gold_output = entity_comparison_prompt_template.render({"entity_pred": {gold_entity_comparison}, "entity_gold": {gold_entity_comparison}})
four_output = entity_comparison_prompt_template.render({"entity_pred": {four_entity_comparison}, "entity_gold": {gold_entity_comparison}})
three_output = entity_comparison_prompt_template.render({"entity_pred": {three_entity_comparison}, "entity_gold": {gold_entity_comparison}})
two_output = entity_comparison_prompt_template.render({"entity_pred": {two_entity_comparison}, "entity_gold": {gold_entity_comparison}})
one_output = entity_comparison_prompt_template.render({"entity_pred": {one_entity_comparison}, "entity_gold": {gold_entity_comparison}})

print("Expecting output: 3 \n")
print(gpt_chat_completion(client, gold_output).choices[0].message.content)

print("\n Expecting output: 4 \n")
print(gpt_chat_completion(client, four_output).choices[0].message.content)

print("\n Expecting output: 3 \n")
print(gpt_chat_completion(client, three_output).choices[0].message.content)

print("\n Expecting output: 2 \n")
print(gpt_chat_completion(client, two_output).choices[0].message.content)

print("\n Expecting output: 1 \n")
print(gpt_chat_completion(client, one_output).choices[0].message.content)  

Expecting output: 3 

```json
{
  "reasoning": "To evaluate the quality of the given documentation against the gold standard, we will compare each element step by step:\n\n1. **Structure and Format**: Both the evaluation documentation and the gold documentation share a similar structure where each attribute of the 'MarketplaceDailySnapshot' is followed by a detailed description.\n\n2. **Content**: The evaluation documentation's content for each field matches perfectly with the gold documentation. Both provide precise and unambiguous explanations for each column.\n\n   - **ID**: The description '{ Contract address }-{# of days since Unix epoch time}' provides necessary context and is precisely the same in both cases.\n   - **Marketplace**: Both describe it as 'The marketplace that this snapshot belongs to.', adequately explaining the field's purpose.\n   - **BlockNumber and Timestamp**: The explanations of when the snapshot is taken, at what block number and timestamp, are clearly detai